In [403]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from itertools import combinations

# В начале идут агрегирующие функции

In [404]:
def user_data_form(events_data, submission_data, sessions_quantile=0.95, leave_quantile=0.9, max_time_month=3, test=False):
    """Функция, чтобы запустить весь цикл функций для тренировочного датасета"""
    total_data_df = total_data(events_data, submission_data, leave_quantile, max_time_month, test,
                               finished_course, timediff, still_learning, first_n_days_df)
    user_data = user_df(total_data_df, sessions_quantile, sessions, avg_time, first_day_features, maxdelta, wrong_only)
    return user_data

In [405]:
def total_data(events_data, submission_data, leave_quantile=0.9, max_time_month=3, test=False, *args):
    """Функция для формирования единого датафрейма полных данных, с необходимыми дополнительными фичами"""
    
    submission_data = submission_data.rename(columns={'submission_status': 'action'})
    total_data = events_data.append(submission_data)
    total_data = pd.get_dummies(total_data) \
        .rename(columns={'action_correct': 'correct', 'action_discovered': 'discovered', 'action_passed': 'passed',
                    'action_started_attempt': 'started_attempt', 'action_viewed': 'viewed', 'action_wrong': 'wrong',
                    'finished_course_True': 'finished_course'})
    for func in args:
        if func == still_learning:
            total_data = func(total_data, leave_quantile=leave_quantile, max_time_month=max_time_month)
        elif func == first_n_days_df:
            total_data = func(total_data, test=test)
        else:    
            total_data = func(total_data)
    return total_data

In [406]:
def user_df(total_data, sessions_quantile=0.95, *args):
    """Функция, чтобы превратить исходный датафрейм в полный датафрейм по юзерам, с добавлением фич"""
    user_data = total_data.groupby('user_id', as_index=False).agg({'correct': 'sum', 'discovered': 'sum', 'passed': 'sum', 
                                                         'started_attempt': 'sum', 'viewed': 'sum', 'wrong': 'sum', 
                                                         'finished_course': 'mean' ,'step_id': lambda x: x.nunique(),
                                                                  'timestamp': 'min'}) \
                    .rename(columns={'step_id': 'steps_passed'})
    user_data['total_actions'] = user_data.correct + user_data.discovered \
        + user_data.passed + user_data.started_attempt + user_data.viewed + user_data.wrong
    user_data['correct_rate'] = user_data.correct / (user_data.correct + user_data.wrong)
    user_data['viewed_rate'] = user_data.viewed / (user_data.total_actions)
    
    for func in args:
        if func == sessions:
            user_data = user_data.merge(func(total_data, sessions_quantile), on='user_id', how='outer')
        else:    
            user_data = user_data.merge(func(total_data), on='user_id', how='outer')
        if func == first_day_features:
            user_data['first_day_actions_rate'] = user_data.total_actions_1_day / user_data.total_actions
            user_data['first_day_correct_rate'] = user_data.correct_1_day / \
                (user_data.correct_1_day + user_data.wrong_1_day)
            user_data['corrects_rate_diff'] = user_data.first_day_correct_rate / user_data.correct_rate
    return user_data.fillna(0)

# После агрегириющих функций - функции, преобразующие "большой" датафрейм

In [407]:
def finished_course(total_data, min_corrects=41):
    """Функция для формирования в исходном датафрейме колонки finished_course. min_corrects - число правильных 
        ответов, с которого курс считается пройденым, по дефолту берется из условия задачи"""
        
    finished_course_data = total_data.query('correct == 1') \
        .groupby(['user_id', 'step_id'], as_index=False) \
        .agg({'correct': 'count'}) \
        .groupby('user_id', as_index=False) \
        .agg({'step_id': 'count'}) \
        .rename(columns={'step_id': 'correct_tasks'})
    finished_course_data['finished_course'] = finished_course_data.correct_tasks >= min_corrects
    total_data = total_data.merge(finished_course_data, on='user_id', how='outer').fillna(False)
    return total_data

In [408]:
def timediff(total_data):
    """Функция чтобы добавить в датафрейм колонку datadiff - показывает разницу во времени между действиями юзера"""
    
    total_data = total_data.sort_values(['user_id', 'timestamp'], ascending=False)
    total_data['previous_action'] = total_data.timestamp.shift(-1)
    total_data['previous_user'] = total_data.user_id.shift(-1)
    total_data['timediff'] = (total_data.timestamp - total_data.previous_action) \
    * (total_data.user_id == total_data.previous_user)
    total_data.drop(['previous_action', 'previous_user'], axis=1)
    return total_data

In [409]:
def still_learning(total_data, leave_quantile=0.9, max_time_month=3):
    """Функция чтобы добавить в датафрейм колонку still_learning - показывает юзеров, которые на текущий момент еще, 
        возможно, учатся. leave_quantile - для определения порогового значения, сколько времени должно пройти между 
        действиями, чтобы юзер считался ушедшим. max_time_month - максимальный период (в месяцах), который используется
        в расчете (чтобы исключить выбросы)"""
        
    max_time = max_time_month * 30 * 24 * 60 * 60
    time_to_leave = total_data.query(f'timediff < {max_time}').groupby('user_id', as_index=False) \
        .agg({'timediff': 'max'}) \
        .timediff \
        .quantile(leave_quantile)
    time_now = total_data.timestamp.max()
    still_learning_time = time_now - time_to_leave
    total_data['still_learning'] = total_data.timestamp >= still_learning_time
    users_still_learning = total_data.query('still_learning == True') \
        .groupby('user_id', as_index=False).agg({'still_learning': 'max'}) \
        .rename(columns={'still_learning': 'users_still_learning'})
    total_data = total_data.merge(users_still_learning, on='user_id', how='outer') \
        .fillna(False).drop(['still_learning'], axis=1) \
        .rename(columns={'users_still_learning': 'still_learning'})
    return total_data

In [410]:
def first_n_days_df(total_data, n=2, test=False):
    """Функция чтобы оставить в датафрейме информацию только
    о первых n днях обучения юзера. n=2, т.к. в тестовом дф данные за 2 дня"""
    time = n * 24 * 60 * 60
    users_start_time = total_data.groupby('user_id', as_index=False) \
        .agg({'timestamp': 'min'}) \
        .rename(columns={'timestamp': 'start_time'})
    total_data = total_data.merge(users_start_time, on='user_id', how='outer')
    if test == False:
        total_data = total_data.query('(timestamp - start_time <= 172800) & ((finished_course == True) | (still_learning == False))')
    return total_data.drop('still_learning', axis=1)    

# Функции, создающие фичи для юзерского датафрейма

In [411]:
def avg_time(total_data):
    """Функция, чтобы по каждому юзеру подготовить фичу 'среднее время выполнения задания'"""
    data_for_attempt_count = total_data.query('(started_attempt == 1) | (correct == 1)')\
                                    .sort_values(['user_id', 'step_id', 'timestamp', 'correct'])
    data_for_attempt_count['attempt_start_time'] = data_for_attempt_count.timestamp.shift(1)
    data_for_attempt_count['time_on_attempt'] = \
        (data_for_attempt_count.timestamp \
         - data_for_attempt_count.attempt_start_time) * data_for_attempt_count.correct
    avg_time_for_attempt = data_for_attempt_count.query('correct == 1')\
        .groupby('user_id', as_index=False).agg({'time_on_attempt': 'mean'}) \
        .rename(columns={'time_on_attempt': 'avg_time_on_attempt'})
    return avg_time_for_attempt[['user_id', 'avg_time_on_attempt']]

In [412]:
def sessions(total_data, session_quatile=0.95):
    """Функция, чтобы по каждому юзеру подготовить фичу 'число сессий'"""
    sessions_delta_min = total_data['timediff'].quantile(session_quatile)
    sessions_data = total_data.query(f'timediff > {sessions_delta_min}')\
        .groupby('user_id', as_index=False) \
        .agg({'timediff': lambda x: x.count()}) \
        .rename(columns={'timediff':'sessions'}) \
        .sort_values('sessions')
    return sessions_data[['user_id', 'sessions']]

In [413]:
def first_day_features(total_data):
    """Функция, чтобы добавить в юзерский датафрейм фичи, связанные с первым днем обучения"""
    total_data_1_day = total_data.query('timestamp - start_time <= 86400')
    user_data_1_day = total_data_1_day.groupby('user_id', as_index=False).agg({'correct': 'sum', 'discovered': 'sum', 'passed': 'sum', 
                                                         'started_attempt': 'sum', 'viewed': 'sum', 'wrong': 'sum', 
                                                        'step_id': lambda x: x.nunique()}) \
            .rename(columns={'correct': 'correct_1_day', 'discovered': 'discovered_1_day', 'passed': 'passed_1_day', 
                    'started_attempt': 'started_attempt_1_day', 'viewed': 'viewed_1_day', 'wrong': 'wrong_1_day', 
                   'step_id': 'steps_passed_1_day'})
    user_data_1_day['total_actions_1_day'] = user_data_1_day.correct_1_day + user_data_1_day.discovered_1_day \
        + user_data_1_day.passed_1_day + user_data_1_day.started_attempt_1_day \
        + user_data_1_day.viewed_1_day + user_data_1_day.wrong_1_day
    return user_data_1_day[['user_id', 'correct_1_day', 'wrong_1_day', 'total_actions_1_day']]

In [414]:
def maxdelta(total_data):
    """Функция, чтобы добавить в юзерский датафрейм фичу "время между первым и последним событием юзера в дф" """

    df = total_data.groupby('user_id', as_index=False)\
        .agg({'timestamp': 'max', 'start_time': 'mean'}).rename(columns={'timestamp': 'maxtime'})
    df['maxdelta'] = df['maxtime'] - df['start_time']
    return df[['user_id', 'maxdelta']]

In [415]:
def wrong_only(total_data):
    """Функция, чтобы добавить в юзерский датафрейм фичу "количество шагов, которые юзер пытался решить, но не решил" """
    
    df = total_data.groupby(['step_id', 'user_id'], as_index=False)\
        .agg({'correct': 'sum', 'wrong': 'sum'}) \
        .query('correct == 0 & wrong != 0')\
        .groupby('user_id', as_index=False) \
        .agg({'wrong': 'count'}) \
        .rename(columns={'wrong': 'wrong_only'})
    return df[['user_id', 'wrong_only']]

# Функции для подбора оптимальной структуры модели. Работают долго

In [416]:
def constant_parametrs(events_data_train, submission_data_train):
    """Функция для определения с помощью оптимальных констант: sessions_quantile, leave_quantile, max_time_month, min_corrects. Функция
    работает очень долго, нужна оптимизация алгоритма, чтобы ей пользоваться на регулярной основе. Текущие параметры подобраны
    с ее помощью"""
    result = {}
    parametrs = {'criterion': ['gini', 'entropy'],
             'n_estimators': range(10, 101, 10), 
             'max_depth': range(4, 9, 1),
             'min_samples_leaf': range(2, 10, 1),
             'min_samples_split': range(10, 40, 5)}
    for sessions_quantile in [85, 90, 95]:
        for leave_quantile in [85, 90, 95]:
            for max_time_month in [2, 5, 8]:
                user_data_train = user_data_form(events_data_train, submission_data_train, sessions_quantile=sessions_quantile/100, 
                                                    leave_quantile=leave_quantile/100, max_time_month=max_time_month, min_corrects=min_corrects)
                X_full = drop_features(user_data_train, drop_parametrs)
                y_full = user_data_train.finished_course
                X_f_train, X_f_test, y_f_train, y_f_test = train_test_split(X_full, y_full, test_size=0.25, random_state=42)
                rf = RandomForestClassifier(random_state=0)
                random_search = RandomizedSearchCV(rf, parametrs, cv=5, n_jobs=-1)
                random_search.fit(X_f_train, y_f_train)
                best_rf = random_search.best_estimator_
                best_rf_score = roc_auc_score(y_f_test, best_rf.predict_proba(X_f_test)[:, 1])
                result[tuple(list([sessions_quantile, leave_quantile, max_time_month, min_corrects]))] = best_rf_score
    result_list = sorted(list(result.items()), key=lambda x: x[1], reverse=True)
    return result_list[0][0]

In [417]:
def features_compilations(X_train, X_test, y_train, y_test):
    """Функция для сравнения возможных комбинаций фич, и выбора оптимальной. На большом количестве фич функция работает
    долго, так что проводить какую-то предобработку, оставлять максимум 5-10 фич"""
    features = list(X_train)
    results = {}
    for i in range(len(features) - 1):
        for combination in combinations(features, i):
            X_train_combination, X_test_combination = X_train, X_test
            for feature in combination:
                X_train_combination = X_train_combination.drop(feature, axis=1)
                X_test_combination = X_test_combination.drop(feature, axis=1)
            rf = RandomForestClassifier(random_state=0)
            random_search = RandomizedSearchCV(rf, parametrs, cv=5, n_jobs=-1)
            random_search.fit(X_train_combination, y_train)
            best_rf = random_search.best_estimator_
            best_rf_score = roc_auc_score(y_test, best_rf.predict_proba(X_test_combination)[:, 1])
            results[combination] = best_rf_score
    res2 = sorted(list(results.items()), key=lambda x: x[1], reverse=True)
    return res2[0][1]

# Функция для удаления фич из датафрейма

In [418]:
def drop_features(df, features):
    """Функция для удаления атрибутов датафрейма"""
    
    for feature in features:
        df = df.drop(feature, axis=1)
    return df

# Тело программы

In [419]:
#Считываем файлы, обработка датафремов, формирование целевого юзерского датафрейма
submission_data_train = pd.read_csv('submissions_data_train.csv')
events_data_train = pd.read_csv('event_data_train.csv')
user_data_train = user_data_form(events_data_train, submission_data_train)

In [420]:
#Сплит на трейн/тестовые датафреймы, которые используются для подбора оптимального набора фич. 
drop_parametrs = ['finished_course', 'user_id', 'steps_passed', \
                                           'viewed', 'started_attempt', 'discovered', 'passed',\
                                           'wrong_1_day', 'total_actions_1_day']
pre_split_df = drop_features(user_data_train, drop_parametrs)
X_f_train, X_f_test, y_f_train, y_f_test = \
train_test_split(pre_split_df, user_data_train.finished_course, test_size=0.1, random_state=42)

In [421]:
#Формирование случайного леса
rf = RandomForestClassifier(random_state=0, class_weight='balanced')
parametrs = {'criterion': ['gini', 'entropy'],
             'n_estimators': range(100, 101, 1), 
             'max_depth': range(6, 7, 1),
             'min_samples_leaf': range(10, 11, 1),
             'min_samples_split': range(30, 31, 1)}
grid_search = GridSearchCV(rf, parametrs, cv=5, n_jobs=-1)
grid_search.fit(X_f_train, y_f_train)
best_rf = grid_search.best_estimator_

In [422]:
#Смотрим результат, обучаем лес на полных данных

print(roc_auc_score(y_f_test, best_rf.predict_proba(X_f_test)[:, 1]))
X_full = drop_features(user_data_train, drop_parametrs)
y_full = user_data_train.finished_course
best_rf.fit(X_full, y_full)

0.8492470749674996


RandomForestClassifier(class_weight='balanced', max_depth=6,
                       min_samples_leaf=10, min_samples_split=30,
                       random_state=0)

In [423]:
#Предобработка тестового датафрейма
submission_data_test = pd.read_csv('submission_data_test.csv')
events_data_test = pd.read_csv('events_data_test.csv')
user_data_test = user_data_form(events_data_test, submission_data_test, test=True)
X_test = drop_features(user_data_test, drop_parametrs)

In [424]:
#Получение результата
y_predicted_prob = best_rf.predict_proba(X_test)
result = pd.DataFrame({'user_id': user_data_test['user_id'], '"is_gone"': y_predicted_prob[:,1]})
result.to_csv(r'C:\Users\Solovov\Dropbox\GoT\Projects\Stepik_ML_contest\test_result.csv', encoding='utf-8')